In [324]:
import sqlite3

In [325]:

# 1️⃣connector
conn = sqlite3.connect("advanced_sql_db_multi.sqlite")
cursor = conn.cursor()

In [326]:

# -----------------------------
# 2️⃣  Departments
cursor.execute("DROP TABLE IF EXISTS departments")
cursor.execute("""
CREATE TABLE departments (
    dept_id INTEGER PRIMARY KEY,
    dept_name TEXT
)
""")
departments_data = [(1, 'IT'), (2, 'HR'), (3, 'Sales')]
cursor.executemany("INSERT INTO departments (dept_id, dept_name) VALUES (?, ?)", departments_data)

In [327]:
# -----------------------------
# 3️⃣  Employees
cursor.execute("DROP TABLE IF EXISTS employees")
cursor.execute("""
CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    name TEXT,
    dept_id INTEGER,
    hire_date DATE,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY(dept_id) REFERENCES departments(dept_id)
)
""")
employees_data = [
    (1, 'Ali', 1, '2023-01-15'),
    (2, 'Sara', 1, '2023-03-10'),
    (3, 'Omar', 1, '2023-05-20'),
    (4, 'Lina', 2, '2022-11-01'),
    (5, 'Hany', 2, '2022-12-12'),
    (6, 'Mona', 3, '2023-02-28'),
    (7, 'Tamer', 3, '2023-04-05'),
    (8, 'Nour', 3, '2023-06-15'),
    (9, 'salama', 1, '2023-05-20'),

]
cursor.executemany("INSERT INTO employees (employee_id, name, dept_id, hire_date) VALUES (?, ?, ?, ?)", employees_data)

In [328]:

# -----------------------------
# 4️⃣  Salaries
cursor.execute("DROP TABLE IF EXISTS salaries")
cursor.execute("""
CREATE TABLE salaries (
    salary_id INTEGER PRIMARY KEY,
    employee_id INTEGER,
    salary_month DATE,
    salary REAL,
    FOREIGN KEY(employee_id) REFERENCES employees(employee_id)
)
""")
salaries_data = [
    (1, 1, '2023-01-01', 10000),
    (2, 1, '2023-02-01', 10500),
    (3, 2, '2023-01-01', 9000),
    (4, 2, '2023-02-01', 9200),
    (5, 3, '2023-01-01', 9000),
    (6, 3, '2023-02-01', 9300),
    (7, 8, '2023-02-01', 9300),
    (8, 9, '2023-02-01', 6000)
]
cursor.executemany("INSERT INTO salaries (salary_id, employee_id, salary_month, salary) VALUES (?, ?, ?, ?)", salaries_data)

In [329]:
# -----------------------------
# 5️⃣  Sales
cursor.execute("DROP TABLE IF EXISTS sales")
cursor.execute("""
CREATE TABLE sales (
    sale_id INTEGER PRIMARY KEY,
    employee_id INTEGER,
    sale_date DATE,
    amount REAL,
    FOREIGN KEY(employee_id) REFERENCES employees(employee_id)
)
""")
sales_data = [
    (1, 6, '2023-01-10', 5000),
    (2, 6, '2023-01-15', 7000),
    (3, 7, '2023-01-20', 6000),
    (4, 7, '2023-02-10', 8000),
    (5, 8, '2023-02-12', 7500),
    (6, 8, '2023-02-15', 6500),
]
cursor.executemany("INSERT INTO sales (sale_id, employee_id, sale_date, amount) VALUES (?, ?, ?, ?)", sales_data)

conn.commit()
print("✅ Multiple tables created with dummy data!\n")

✅ Multiple tables created with dummy data!



In [330]:
# -----------------------------
#second salary per each employee
# 6️⃣ ROW_NUMBER
print("--- ROW_NUMBER per Department ---")
query_row_number = """
SELECT *
FROM (
    SELECT e.employee_id, e.name, d.dept_name, s.salary,
           ROW_NUMBER() OVER (PARTITION BY e.employee_id ORDER BY s.salary DESC) AS row_num
    FROM employees e
    JOIN salaries s ON e.employee_id = s.employee_id
    JOIN departments d ON e.dept_id = d.dept_id
) t
--where row_num = 2
ORDER BY dept_name, salary DESC
"""
for row in cursor.execute(query_row_number):
    print(row)

--- ROW_NUMBER per Department ---
(1, 'Ali', 'IT', 10500.0, 1)
(1, 'Ali', 'IT', 10000.0, 2)
(3, 'Omar', 'IT', 9300.0, 1)
(2, 'Sara', 'IT', 9200.0, 1)
(2, 'Sara', 'IT', 9000.0, 2)
(3, 'Omar', 'IT', 9000.0, 2)
(9, 'salama', 'IT', 6000.0, 1)
(8, 'Nour', 'Sales', 9300.0, 1)


In [331]:

# -----------------------------
# 7️⃣ RANK (with gap)
print("\n--- RANK per Department ---")
query_rank = """
SELECT *
FROM (
    SELECT e.employee_id, e.name, d.dept_name, e.dept_id,s.salary,
           RANK() OVER (PARTITION BY e.dept_id ORDER BY s.salary DESC) AS rnk
    FROM employees e
    JOIN salaries s ON e.employee_id = s.employee_id
    JOIN departments d ON e.dept_id = d.dept_id
) t
ORDER BY dept_name, salary DESC
"""
for row in cursor.execute(query_rank):
    print(row)


--- RANK per Department ---
(1, 'Ali', 'IT', 1, 10500.0, 1)
(1, 'Ali', 'IT', 1, 10000.0, 2)
(3, 'Omar', 'IT', 1, 9300.0, 3)
(2, 'Sara', 'IT', 1, 9200.0, 4)
(2, 'Sara', 'IT', 1, 9000.0, 5)
(3, 'Omar', 'IT', 1, 9000.0, 5)
(9, 'salama', 'IT', 1, 6000.0, 7)
(8, 'Nour', 'Sales', 3, 9300.0, 1)


In [332]:

# -----------------------------
# 8️⃣ DENSE_RANK (without gap)
print("\n--- DENSE_RANK per Department ---")
query_dense_rank = """
SELECT *
FROM (
    SELECT e.employee_id, e.name, d.dept_name,e.dept_id, s.salary,
           DENSE_RANK() OVER (PARTITION BY e.dept_id ORDER BY s.salary DESC) AS dense_rnk
    FROM employees e
    JOIN salaries s ON e.employee_id = s.employee_id
    JOIN departments d ON e.dept_id = d.dept_id
) t
ORDER BY dept_name, salary DESC
"""
for row in cursor.execute(query_dense_rank):
    print(row)


--- DENSE_RANK per Department ---
(1, 'Ali', 'IT', 1, 10500.0, 1)
(1, 'Ali', 'IT', 1, 10000.0, 2)
(3, 'Omar', 'IT', 1, 9300.0, 3)
(2, 'Sara', 'IT', 1, 9200.0, 4)
(2, 'Sara', 'IT', 1, 9000.0, 5)
(3, 'Omar', 'IT', 1, 9000.0, 5)
(9, 'salama', 'IT', 1, 6000.0, 6)
(8, 'Nour', 'Sales', 3, 9300.0, 1)


In [333]:
# -----------------------------
# 9️⃣ LAG (history of employee salary )
print("\n--- LAG Salary per Department ---")
query_lag = """
SELECT *
FROM (
    SELECT e.employee_id, e.name, e.dept_id, d.dept_name, s.salary,
           LAG(s.salary, 1) OVER (PARTITION BY e.employee_id ORDER BY s.salary asc) AS prev_salary
    FROM employees e
    JOIN salaries s ON e.employee_id = s.employee_id
    JOIN departments d ON e.dept_id = d.dept_id
) t

"""
for row in cursor.execute(query_lag):
    print(row)


--- LAG Salary per Department ---
(1, 'Ali', 1, 'IT', 10000.0, None)
(1, 'Ali', 1, 'IT', 10500.0, 10000.0)
(2, 'Sara', 1, 'IT', 9000.0, None)
(2, 'Sara', 1, 'IT', 9200.0, 9000.0)
(3, 'Omar', 1, 'IT', 9000.0, None)
(3, 'Omar', 1, 'IT', 9300.0, 9000.0)
(8, 'Nour', 3, 'Sales', 9300.0, None)
(9, 'salama', 1, 'IT', 6000.0, None)


**Fix Issue of duplication here ?**

In [334]:
# write your code here


--- LAG Salary per Department ---
(1, 'Ali', 'IT', 10500.0, 2, 10000.0)
(3, 'Omar', 'IT', 9300.0, 2, 9000.0)
(2, 'Sara', 'IT', 9200.0, 2, 9000.0)


In [335]:

# -----------------------------
# 10️⃣ LEAD
print("\n--- LEAD Salary each employe per Department ---")
query_lead = """
SELECT *
FROM (
    SELECT e.employee_id, e.name, d.dept_name, s.salary,
           LEAD(s.salary, 1) OVER (PARTITION BY e.employee_id ORDER BY s.salary asc) AS next_salary
    FROM employees e
    JOIN salaries s ON e.employee_id = s.employee_id
    JOIN departments d ON e.dept_id = d.dept_id
) t

"""
for row in cursor.execute(query_lead):
    print(row)


--- LEAD Salary per Department ---
(1, 'Ali', 'IT', 10000.0, 10500.0)
(1, 'Ali', 'IT', 10500.0, None)
(2, 'Sara', 'IT', 9000.0, 9200.0)
(2, 'Sara', 'IT', 9200.0, None)
(3, 'Omar', 'IT', 9000.0, 9300.0)
(3, 'Omar', 'IT', 9300.0, None)
(8, 'Nour', 'Sales', 9300.0, None)
(9, 'salama', 'IT', 6000.0, None)


**Fix Issue of duplication here ?**

In [335]:
#write your code here

In [336]:


# -----------------------------
# 11️⃣ SUM() OVER (Running Total) per each deparment
print("\n--- SUM() OVER (Running Total per Department) ---")
query_sum = """
SELECT *
FROM (
    SELECT e.employee_id, e.name, d.dept_name, s.salary,
           SUM(s.salary) OVER (PARTITION BY e.dept_id ORDER BY s.salary DESC) AS running_total
    FROM employees e
    JOIN salaries s ON e.employee_id = s.employee_id
    JOIN departments d ON e.dept_id = d.dept_id
) t
ORDER BY dept_name, salary DESC
"""
for row in cursor.execute(query_sum):
    print(row)

conn.close()



--- SUM() OVER (Running Total per Department) ---
(1, 'Ali', 'IT', 10500.0, 10500.0)
(1, 'Ali', 'IT', 10000.0, 20500.0)
(3, 'Omar', 'IT', 9300.0, 29800.0)
(2, 'Sara', 'IT', 9200.0, 39000.0)
(2, 'Sara', 'IT', 9000.0, 57000.0)
(3, 'Omar', 'IT', 9000.0, 57000.0)
(9, 'salama', 'IT', 6000.0, 63000.0)
(8, 'Nour', 'Sales', 9300.0, 9300.0)
